# Direct Stiffness Method

The direct stiffness method is a displacement-based structural analysis method; that is, the primary unkowns are displacement-like quantities i.e. translations and rotations. Force-like quantities are obtained in post-processing. For a linear problem, this method results in a system of linear algebraic equations.

## Example Problem 1: Frame with rectangular cross-sections

In the cell below, you import the solver to this notebook, and initiate the process of defining all the relevant geometric and material properties of the structure.

In [61]:
# --- Import the solver from /src directiory (Don't touch!) ---
import sys
import os
project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(os.path.join(project_path, "src"))
from direct_stiffness_method.direct_stiffness_method import Structure, BoundaryConditions, Solver, PostProcessing

import numpy as np

# Define nodes and their coordinates [#, x, y, z]
nodes = {
    0: [0, 0.0, 0.0, 10.0],
    1: [1, 15.0, 0.0, 10.0],
    2: [2, 15.0, 0.0, 0.0]
}

# Define the connectivity [node i, node j]
elements = [
    [0, 1],
    [1, 2]
]

# Elements properties
element_properties = {
    0: {"b": 0.5, "h": 1.0, "E": 1000, "nu": 0.3},
    
    1: {"b": 1.0, "h": 0.5, "E": 1000, "nu": 0.3}
}


# Run class
structure = Structure(nodes, elements, element_properties)

In the cell below, you get a summary for the properties you entered.

In [64]:
# Display summary for the geometric and material properties per element
structure.display_summary()

--- Structure Summary ---
Number of Elements: 2
Elasticity Modulus (E):
  Element 0: 1000
  Element 1: 1000

Poisson's Ratio (nu):
  Element 0: 0.3
  Element 1: 0.3

--- Element Properties ---
Element 1:
  Length: 15.0000
  Area (A): 0.5000
  Moment of Inertia Iy: 0.0104
  Moment of Inertia Iz: 0.0417
  Polar Moment of Inertia J: 0.0521
  Node 1: (0.0, 0.0, 10.0), Node 2: (15.0, 0.0, 10.0)

Element 2:
  Length: 10.0000
  Area (A): 0.5000
  Moment of Inertia Iy: 0.0417
  Moment of Inertia Iz: 0.0104
  Polar Moment of Inertia J: 0.0521
  Node 1: (15.0, 0.0, 10.0), Node 2: (15.0, 0.0, 0.0)

--- Connectivity Matrix ---
Element 1: [0 1]
Element 2: [1 2]

Global Node Numbering:
Global Node 0: Coordinates (0.0, 0.0, 10.0)
Global Node 1: Coordinates (15.0, 0.0, 10.0)
Global Node 2: Coordinates (15.0, 0.0, 0.0)

* * * * * * * * * *


In the below cell, you define the loads and the boundary conditions.

In [67]:
# Define externally applied loads [#, Fx, Fy, Fz, Mx, My, Mz]

loads = {
    0: [0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    1: [1, 0.1, 0.05, -0.07, 0.05, -0.1, 0.25],
    2: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
}

# Define supports [#, ux, uy, uz, theta_x, theta_y, theta_z]
# 1 means constrained dof
# 0 means free dof

supports = {
    0: [0, 1, 1, 1, 1, 1, 1],
    1: [1, 0, 0, 0, 0, 0, 0],
    2: [2, 1, 1, 1, 0, 0, 0]
}

bc = BoundaryConditions(loads, supports)

In [69]:
# Compute and print the external load vector
bc.compute_global_load_vector()
bc.print_global_load_vector()

# Summarize and print boundary conditions
bc.summarize_boundary_conditions()


--- External Load Vector ---
[[ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.1 ]
 [ 0.05]
 [-0.07]
 [ 0.05]
 [-0.1 ]
 [ 0.25]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]]

--- Boundary Conditions ---
Node 0: Constraints [1, 1, 1, 1, 1, 1]
Node 1: Constraints [0, 0, 0, 0, 0, 0]
Node 2: Constraints [1, 1, 1, 0, 0, 0]


Now, in the below cell, the geometric boundary conditions are applied, then the problem is solved.

In [72]:
solver = Solver(structure, bc)
U_global = solver.solve()
R_global = solver.compute_reactions(U_global)


--- Computed Displacements ---
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 2.84049953e-03]
 [ 1.43541318e+00]
 [-1.30609178e-03]
 [-1.26072079e-01]
 [-1.67293339e-02]
 [ 1.66041318e-01]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-1.52275937e-01]
 [ 8.79074190e-03]
 [ 1.66041318e-01]]

--- Computed Reactions ---
[[-0.09468332]
 [-0.02816345]
 [ 0.00469541]
 [ 0.16836549]
 [-0.02359799]
 [-0.67245177]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.00531668]
 [-0.02183655]
 [ 0.06530459]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]


Now, we post-process by calculating the internal forces per element in local coordinates.

In [76]:
# Create post-processing object
post_processing = PostProcessing(structure, U_global)

# Compute internal forces
post_processing.compute_internal_forces()

# Print internal forces
post_processing.print_internal_forces()


--- Internal Forces in Local Coordinates ---
Element 1:
[-0.09468332 -0.02816345  0.00469541  0.16836549 -0.02359799 -0.67245177
  0.09468332  0.02816345 -0.00469541 -0.16836549 -0.04683318  0.25      ]
Element 2:
[ 6.53045890e-02 -5.31668247e-03  2.18365490e-02 -1.02529531e-17
 -2.18365490e-01 -5.31668247e-02 -6.53045890e-02  5.31668247e-03
 -2.18365490e-02  1.02529531e-17  2.86968227e-16  1.15548791e-18]


## Example Problem 2: Frame with circular cross-sections

In the cell below, you import the solver to this notebook, and initiate the process of defining all the relevant geometric and material properties of the structure.

In [ ]:
# --- Import the solver from /src directiory (Don't touch!) ---
import sys
import os
project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(os.path.join(project_path, "src"))
from direct_stiffness_method.direct_stiffness_method import Structure, BoundaryConditions, Solver

import numpy as np

# Define nodes and their coordinates [#, x, y, z]
nodes = {
    0: [0, 0.0, 0.0, 0.0],
    1: [1, -5.0, 1.0, 10.0],
    2: [2, -1.0, 5.0, 13.0],
    3: [3, -3.0, 7.0, 11.0],
    4: [4, 6.0, 9.0, 5.0]
}

# Define the connectivity [node i, node j]
elements = [
    [0, 1],
    [1, 2],
    [2, 3],
    [2, 4]
]

# Elements properties
element_properties = {
    0: {"r": 1.0, "E": 500, "nu": 0.3},
    
    1: {"r": 1.0, "E": 500, "nu": 0.3},

    2: {"r": 1.0, "E": 500, "nu": 0.3},
    
    3: {"r": 1.0, "E": 500, "nu": 0.3},

}

# Run class
structure = Structure(nodes, elements, element_properties)

In the cell below, you get a summary for the properties you entered.

In [ ]:
# Display summary for the geometric and material properties per element
structure.display_summary()

In the below cell, you define the loads and the boundary conditions.

In [ ]:
# Define externally applied loads [#, Fx, Fy, Fz, Mx, My, Mz]

loads = {
    0: [0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    1: [1, 0.1, -0.05, -0.075, 0.0, 0.0, 0.0],
    2: [2, 0.0, 0.0, 0.0, 0.5, -0.1, 0.3],
    3: [3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    4: [4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
}

# Define supports [#, ux, uy, uz, theta_x, theta_y, theta_z]
# 1 means constrained dof
# 0 means free dof

supports = {
    0: [0, 0, 0, 1, 0, 0, 0],
    1: [1, 0, 0, 0, 0, 0, 0],
    2: [2, 0, 0, 0, 0, 0, 0],
    3: [2, 1, 1, 1, 1, 1, 1],
    4: [2, 1, 1, 1, 0, 0, 0]    
}

bc = BoundaryConditions(loads, supports)

In [ ]:
# Compute and print the external load vector
bc.compute_global_load_vector()
bc.print_global_load_vector()

# Summarize and print boundary conditions
bc.summarize_boundary_conditions()

Now, in the below cell, the geometric boundary conditions are applied, then the problem is solved.

In [ ]:
solver = Solver(structure, bc)
U_global = solver.solve()
R_global = solver.compute_reactions(U_global)